In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 한글폰트설정

In [ ]:
%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

mpl.rcParams['axes.unicode_minus'] = False

fontpath = "../input/nanum34/NanumSquareR.ttf"
fontprop = fm.FontProperties(fname=fontpath,size=12)

# Introduction

심혈관 질환 (CVD)은 전 세계적으로 사망 원인 1 위이며 매년 1,790 만 명의 목숨을 앗아 가고 있으며, 이는 전 세계적으로 전체 사망의 31 %를 차지합니다.

심부전은 CVD로 인해 발생하는 일반적인 이벤트이며이 데이터 세트에는 심부전으로 인한 사망률을 예측하는 데 사용할 수있는 12 가지 기능이 포함되어 있습니다.

대부분의 심혈관 질환은 흡연, 건강에 해로운 식습관 및 비만, 신체 활동 불능 및 인구 전체의 전략을 사용하여 알코올의 유해한 사용과 같은 행동 위험 요소를 해결함으로써 예방할 수 있습니다.

심혈관 질환이 있거나 심혈관 위험이 높은 사람 (고혈압, 당뇨병, 고지혈증 또는 이미 확립 된 질병과 같은 하나 이상의 위험 요인이 존재하기 때문에)은 기계 학습 모델이 큰 도움이 될 수있는 조기 발견 및 관리가 필요합니다.

# Data Exploration & Visualisation

데이터 세트는 Giuseppe Jurman의 Davide Chicco에서 얻었습니다. 머신 러닝은 혈청 크레아티닌과 박출 분획만으로 심부전 환자의 생존을 예측할 수 있습니다. BMC Medical Informatics and Decision Making 20, 16 (2020).

# 라이브러리 Import

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import seaborn as sns
import plotly.express as px
import plotly.figure_factory as ff
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings("ignore")

# Reading Data

심부전으로 인한 사망률을 예측하는 데 사용할 수있는 12 가지 기능..

변수 Variables
* age : 나이
* anaemia : 빈혈 (없으면 0, 있으면 1)
* creatinine_phosphokinase : 혈액 내 CPK 효소 수치 (mcg / L)
(cpk : 근육 수축에 관여하는 에너지원으로 사용되는 물질이 활성화되는데 필요한 효소)
* diabetes : 당뇨병, 환자에게 당뇨병이있는 경우 (없는 경우 0, 존재하는 경우 1)
* ejection_fraction : 좌심실이 수축 할 때마다 심장에서 내보내는 혈액의 비율 (백분율)
* high_blood_pressure : 고혈압. 환자에게 고혈압이있는 경우 (없는 경우 0, 존재하는 경우 1)
* platelets : 혈액 내 혈소판 수 (kiloplatelets / mL)
* serum_creatinine : 혈중 혈청 크레아티닌 수치 (mg / dL)
( 혈청크레아티닌 : 근육 속에 존재하며 에너지원이 되는 크레아틴의 대사산물. )
* serum_sodium : 혈중 혈청 나트륨 수치 (mEq / L)
* sex : 성별 (여성 0, 남성 1)
* smoking : 흡연 (안하면 0, 하면 1)
* time : 후속 작업 기간 (일)
* DEATH_EVENT : 환자가 추적 기간 동안 사망 한 경우. (비사망 0, 사망 1)

In [ ]:
data=pd.read_csv("/kaggle/input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv")
data

In [ ]:
heart=Counter(data['DEATH_EVENT'])
classes=[]
count=[]   #list to store no of laels of each class
for i in heart.keys():
    classes.append(i)
    count.append(heart[i])
colors = ["#cf3434", "#ff7d7d"]

plt.pie(
    count,
    labels = classes,
    shadow = False,
    colors = colors,
    startangle=80,
    autopct='%1.1f%%'
)
plt.axis('equal')
plt.tight_layout()
plt.title("환자 중 사망한 비율", fontproperties=fontprop)
plt.show()
# fig = px.pie(df, names='output', title='Percentage of person with heart disease present in the dataset')
# fig.show()

# Distribution of Features

환자들은 대부분 40세-70세, 남성의 비율이 조금 더 많았고, 수축할 때 심장에서 내보내는 혈액의 비율이 채 40%가 되지 않음 (정상의 경우 60-70%) 

In [ ]:
fig=plt.figure(figsize=(16,22))

fig = plt.subplot(431)
#data['age'].value_counts().plot(kind='bar',label='age',color="lightpink")
sns.distplot(data['age'], color="red", kde= True,label='Blood Pressure')
plt.xlabel('age')
plt.title ('나이', fontproperties=fontprop)
plt.legend() # 범례추가

fig = plt.subplot(4,3,2)
data['anaemia'].value_counts().plot(kind='bar',label='anaemia', color="brown")
plt.xlabel('anaemia')
plt.title ('빈혈증', fontproperties=fontprop)
plt.legend()

fig = plt.subplot(4,3,3)
#sns.kdeplot(data['creatinine_phosphokinase'],shade=True,label='creatinine_phosphokinase')
sns.distplot(data['creatinine_phosphokinase'], color="red", kde= True,label='Blood Pressure')
plt.title ('혈액 내 CPK 효소 수치 (mcg / L)', fontproperties=fontprop)
plt.legend()

fig = plt.subplot(434)
#data['age'].value_counts().plot(kind='bar',label='age',color="lightpink")
sns.distplot(data['ejection_fraction'], color="red", kde= True,label='Blood ejection_fraction')
plt.xlabel('ejection_fraction')
plt.title ('수축 할 때마다 심장을 떠나는 혈액의 비율 (백분율)', fontproperties=fontprop)
plt.legend() # 범례추가

fig = plt.subplot(4,3,5)
data['high_blood_pressure'].value_counts().plot(kind='bar',label='high_blood_pressure', color="brown")
plt.xlabel('high_blood_pressure')
plt.title ('고혈압', fontproperties=fontprop)
plt.legend()

fig = plt.subplot(4,3,6)
sns.kdeplot(data['platelets'],shade=True,label='platelets', color='red')
plt.title ('혈액 내 혈소판', fontproperties=fontprop)
plt.legend()

fig = plt.subplot(4,3,7)
#sns.kdeplot(data['serum_creatinine'],shade=True,label='serum_creatinine')
sns.distplot(data['serum_creatinine'], color="red", kde= True,label='Blood Pressure')
plt.title ('혈중 혈청 크레아티닌 수치 (mg / dL)', fontproperties=fontprop)
plt.legend()

fig = plt.subplot(4,3,8)
#sns.kdeplot(data['serum_sodium'],shade=True,label='serum_sodium')
sns.distplot(data['serum_sodium'], color="red", kde= True,label='Blood Pressure')
plt.title ('혈중 혈청 나트륨 수치 (mEq / L)', fontproperties=fontprop)
plt.legend()

fig = plt.subplot(4,3,9)
data['sex'].value_counts().plot(kind='bar',label='sex', color="brown")
plt.xlabel('sex')
plt.title ('성별', fontproperties=fontprop)
plt.legend()

fig = plt.subplot(4,3,10)
data['smoking'].value_counts().plot(kind='bar',label='smoking', color="brown")
plt.xlabel('smoking')
plt.title ('흡연', fontproperties=fontprop)
plt.legend()

fig = plt.subplot(4,3,11)
sns.kdeplot(data['time'],shade=True,label='time')
plt.legend()

plt.show()

# 통계 분석

# 사망과 비사망 평균비교
사망 유뮤별로 데이터의 평균을 내서 비교한다.
* **사망시 평균나이가 65세로 비사망보다 약 7살 높음**
* 빈혈은 비슷함
* **혈액 내 cpk효소수치가 670mcg/L으로 약 130mcg/L 높음**
* 당뇨병의 여부는 비슷함
* **수축할 때마다 심장에서 내보내는 혈액의 비율이 33퍼센트로 7% 더 적음**
* **고혈압의 경우 고혈압이 있는 사람의 비율이 10%정도 많음**
* 혈액 내 혈소판 수는 유사함
* **혈중 혈청 크레아티닌 수치는 1.8mg/dL 로 약 0.65mg/dL정도 높음**
* 혈중 혈청 나트륨 수치는 비슷함
* 성별은 비슷함
* 작업기간 중.. 사망평균일은 70일

즉, 평균나이가 높을수록, 혈액 내 cpk 효소수치가 높을수록, 수축 시 심장에서 내보내는 혈액의 비율이 적을수록, 고혈압이 있을수록, 혈중혈청 크레아티닌 수치가 높을수록 심부전 중 사망확률이 증가한다는 것을 알 수 있음

In [ ]:
# 모든 피험자평균도?
# 사망과 비사망 평균
mean_data=data.groupby(data['DEATH_EVENT']).mean()
mean_data

# 사망률 확인
## - 나이(age)

환자들의 나이는 40~95세이다.
* 대부분의 환자는 50대(27.5%)와 60대(31.1%)이다.
* **환자의 나이가 증가할 수록 사망률 증가**

In [ ]:
fig=plt.figure(figsize=(25, 20))

# 나이별 환자 수
#age_data=data.groupby(data['age'])['DEATH_EVENT'].count() 
age_death = data['DEATH_EVENT'].groupby(data['age']).count()
fig = plt.subplot(431)
#sns.kdeplot(age_death,shade=True,label='Age')
age_death.plot(label='count',color="lightpink")
plt.legend()

# 나이별 사망자 수
age_death2 = data['DEATH_EVENT'].groupby(data['age']).sum()
fig = plt.subplot(431)
age_death2.plot(label='death',color="red")
plt.title ('나이별 환자 및 사망자 수', fontproperties=fontprop)
plt.legend()

# 나이별 사망자 비율
age_death3 = data['DEATH_EVENT'].groupby(data['age']).mean()
fig = plt.subplot(432)
age_death3.plot(label='death_mean',color='#e80505')
plt.title ('나이별 사망자 비율', fontproperties=fontprop)
plt.legend()

plt.show()

In [ ]:
# 나이별 사망자 수 차트
sns.countplot('age', hue = 'DEATH_EVENT', data = data)
plt.title('Age vs Survived')
plt.show()

## -혈청 크레아티닌(serum_creatinine)
혈청 크레아티닌 검사는 혈액 내 크레아티닌 수치를 측정하고 신장이 얼마나 잘 여과되는지 (사구체 여과율) 추정치를 제공.
혈중 크레아티닌의 정상 범위는 0.84 ~ 1.21mg / dL

* 환자들의 혈청 크레아티닌 범위는 0.5~9.4mg/dL이며 평균은 1.394이다.
* 혈청 크레아티닌 (mg / dL)의 평균 값은 비 사망 사건의 경우 1.18이고 사망 사건의 경우 1.84로 사망의 경우 약 0.65mg/dL정도 높음

* **혈청 크레아티닌의 수치가 높을수록(약 2이상이 되면) 사망률이 증가**

In [ ]:
fig=plt.figure(figsize=(25, 20))

# 혈청 크레아티닌 별 사망률
cpk_death=data['DEATH_EVENT'].groupby(data['serum_creatinine']).mean()
fig = plt.subplot(432)
cpk_death.plot(label='death',color="red")
plt.title ('혈청 크레아티닌 별 사망률', fontproperties=fontprop)
plt.legend()

#fig = plt.subplot(4,3,2)
#mean_data.plot('age','serum_creatinine',color="red")
#plt.title ('나이별 혈청 크레아티닌 수치', fontproperties=fontprop)
#plt.legend()

# 혈청 크레아티닌수치별 환자 수
#age_data=data.groupby(data['age'])['DEATH_EVENT'].count() 
age_death = data['DEATH_EVENT'].groupby(data['serum_creatinine']).count()
fig = plt.subplot(431)
#sns.kdeplot(age_death,shade=True,label='Age')
age_death.plot(label='count',color="lightpink")
plt.legend()

# 혈청 크레아티닌수치별 사망자 비율
age_death3 = data['DEATH_EVENT'].groupby(data['serum_creatinine']).mean()
fig = plt.subplot(431)
age_death3.plot(label='death_mean',color='#e80505')
plt.title ('혈청 크레아티닌수치 사망자 비율', fontproperties=fontprop)
plt.legend()

## - 성별별 사망률
성별에 의한 사망률의 차이는 없다

In [ ]:
import matplotlib.pyplot as plt
f,ax = plt.subplots(1, 2, figsize = (10, 5))
data['DEATH_EVENT'][data['sex'] == 1].value_counts().plot.pie(autopct = '%1.1f%%', ax = ax[0], shadow = True)
data['DEATH_EVENT'][data['sex'] == 0].value_counts().plot.pie(autopct = '%1.1f%%', ax = ax[1], shadow = True)
ax[0].set_title('DEATH_EVENT (Male)')
ax[1].set_title('DEATH_EVENT (Female)')
plt.show()

In [ ]:
data.info()

In [ ]:
import seaborn as sns
import pandas as pd

data.isnull().sum()

# 상관 분석
## 1. 상관 분석을 위한 상관계수 구하고 저장

In [ ]:
data_corr = data.corr(method = 'pearson')
data_corr

data_corr.to_csv('heart-failure-clinical-data_corr.csv', index = False)

## 2. 상관계수 확인
- death_event와의 상관관계
- 나이, 빈혈, cpk, 고혈압, serum_creatinine 은 양의 상관관계
- 당뇨(diabetes),ejection_fraction, 혈소판수(platelets), 나트륨수치, 성별, 흡연 등은 음의 상관관계

In [ ]:
data_corr

In [ ]:
data['DEATH_EVENT'].corr(data['ejection_fraction']) # death_event와 ejection fraction의 상관관계구함

# 결과 시각화
## 1. 산점도로 상관분석 시각화

In [ ]:
sns.pairplot(data, hue='DEATH_EVENT')
plt.show()

## 2. 두 변수로 상관관계 시각화
사망(death_event)와 나머지 요소들과의 상관관계를 시각화함

In [ ]:
sns.set_palette(sns.color_palette(colors))

sns.catplot(x='DEATH_EVENT', y='age', hue='sex', data=data,kind='point')
plt.title('나이와 사망의 상관관계', fontproperties=fontprop)

sns.catplot(x='DEATH_EVENT', y='creatinine_phosphokinase', hue='sex', data=data,kind='point')
plt.title('혈액 내 CPK와 사망의 상관관계', fontproperties=fontprop)

sns.catplot(x='DEATH_EVENT', y='diabetes', hue='sex', data=data,kind='point')
plt.title('당뇨와 사망의 상관관계', fontproperties=fontprop)

sns.catplot(x='DEATH_EVENT', y='ejection_fraction', hue='sex', data=data,kind='point')
plt.title('심장에서 내보내는 혈액의 비율과 사망의 상관관계', fontproperties=fontprop)

sns.catplot(x='DEATH_EVENT', y='high_blood_pressure', hue='sex', data=data,kind='point')
plt.title('고혈압과 사망의 상관관계', fontproperties=fontprop)

sns.catplot(x='DEATH_EVENT', y='platelets', hue='sex', data=data,kind='point')
plt.title('혈소판 수와 사망의 상관관계', fontproperties=fontprop)

sns.catplot(x='DEATH_EVENT', y='serum_creatinine', hue='sex', data=data,kind='point')
plt.title('혈청 크레아티닌과 사망의 상관관계', fontproperties=fontprop)

sns.catplot(x='DEATH_EVENT', y='serum_sodium', hue='sex', data=data,kind='point')
plt.title('혈청 나트륨수치와 사망의 상관관계', fontproperties=fontprop)

sns.catplot(x='DEATH_EVENT', y='smoking', hue='sex', data=data,kind='point')
plt.title('흡연과 사망의 상관관계', fontproperties=fontprop)

plt.show()